# Import packages

In [18]:
import os
import sys
sys.path.append('/g/data/fu5/eac/20year_run_Junde/jl4424/KC_GS/Figures')
import cmaps
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from xarrayMannKendall import Mann_Kendall_test
from dask.distributed import Client
from astropy import convolution
import PUG_functions as PF
rcParams['font.family'] = 'sans-serif'
directory   = '/g/data/fu5/eac/20year_run_Junde/jl4424/KC_GS'

# Read large scale wind stress data

In [ ]:
dataset0      = sio.loadmat(directory+'/Figures/Figure2_mean.mat')
dataset1      = xr.open_dataset(directory+'/ERA5/ERA5_monthly_Tx_LS.nc')
dataset2      = xr.open_dataset(directory+'/ERA5/ERA5_monthly_Ty_LS.nc')
dataset3      = xr.open_dataset(directory+'/ERA5/ERA5_monthly_curlZ_LS.nc')
dataset4      = xr.open_dataset(directory+'/ERA5/ERA5_monthly_u10_LS.nc')
dataset5      = xr.open_dataset(directory+'/ERA5/ERA5_monthly_curlZ_trends_LS.nc')
sst_mean      = dataset0['oisst_mean'][:,:-1]
lon           = np.tile(dataset1.lon, (len(dataset1.lat),1)).transpose()
lat           = np.tile(dataset1.lat, (len(dataset1.lon),1))
Tx_mean       = dataset1.Tx.transpose().mean(axis=2).values
Ty_mean       = dataset2.Ty.transpose().mean(axis=2).values
curlZ_mean    = dataset3.curlZ.transpose().mean(axis=2).values*1e7
u10           = np.nanmean(dataset4.u10.values,axis=0).transpose()
u10_mean      = np.fliplr(np.concatenate((u10[1199:,160:361],u10[:681,160:361]),axis=0))
curlZ_trend   = dataset5.trend.transpose().values*1e8*120
curlZ_trend_p = dataset5.p.transpose().values
Tx_mean[np.where(np.isnan(sst_mean))] = np.nan
Ty_mean[np.where(np.isnan(sst_mean))] = np.nan
curlZ_mean[np.where(np.isnan(sst_mean))] = np.nan
u10_mean[np.where(np.isnan(sst_mean))] = np.nan
curlZ_trend[np.where(np.isnan(sst_mean))] = np.nan
curlZ_trend_p[np.where(np.isnan(sst_mean))] = np.nan
curlZ_trend_p[np.where(curlZ_trend_p > 0.05)] = np.nan

# Read zonal mean wind

In [20]:
dataset6                   = sio.loadmat(directory+'/Figures/Figure5_lat_wind.mat')
lat0                       = dataset6['lat0'][:,0]
# Mean state
mean_u10_NH                = dataset6['mean_u10_NH'][0,:]
mean_u10_KC                = dataset6['mean_u10_KC'][0,:]
mean_u10_GS                = dataset6['mean_u10_GS'][0,:]
mean_curlZ_NH              = dataset6['mean_curlZ_NH'][0,:]
mean_curlZ_KC              = dataset6['mean_curlZ_KC'][0,:]
mean_curlZ_GS              = dataset6['mean_curlZ_GS'][0,:]
latitudes_0_u10_NH         = dataset6['latitudes_0_u10_NH'][0,:]
latitudes_0_u10_KC         = dataset6['latitudes_0_u10_KC'][0,:]
latitudes_0_u10_GS         = dataset6['latitudes_0_u10_GS'][0,:]
latitudes_0_curlZ_NH       = dataset6['latitudes_0_curlZ_NH'][0,:]
latitudes_0_curlZ_KC       = dataset6['latitudes_0_curlZ_KC'][0,:]
latitudes_0_curlZ_GS       = dataset6['latitudes_0_curlZ_GS'][0,:]
u10_zonal_NH               = dataset6['u10_zonal_NH'][:,6:-5]
u10_zonal_KC               = dataset6['u10_zonal_KC'][:,6:-5]
u10_zonal_GS               = dataset6['u10_zonal_GS'][:,6:-5]
# Trend
mean_u10_trend_NH          = dataset6['mean_u10_trend_NH'][0,:]*120
mean_u10_trend_KC          = dataset6['mean_u10_trend_KC'][0,:]*120
mean_u10_trend_GS          = dataset6['mean_u10_trend_GS'][0,:]*120
latitudes_0_u10_trend_NH   = dataset6['latitudes_0_u10_trend_NH'][0,:]
latitudes_0_u10_trend_KC   = dataset6['latitudes_0_u10_trend_KC'][0,:]
latitudes_0_u10_trend_GS   = dataset6['latitudes_0_u10_trend_GS'][0,:]
latitudes_0_curlZ_trend_NH = dataset6['latitudes_0_curlZ_trend_NH'][0,:]
latitudes_0_curlZ_trend_KC = dataset6['latitudes_0_curlZ_trend_KC'][0,:]
latitudes_0_curlZ_trend_GS = dataset6['latitudes_0_curlZ_trend_GS'][0,:]
time_start = np.datetime64('1993-07')
time_end   = np.datetime64('2023-08')
t_range    = PF.time_range(time_start,time_end,'monthly','np64')
###################################################################################################
lat_wind_mean      = np.array((latitudes_0_u10_NH,latitudes_0_u10_KC,latitudes_0_u10_GS))
ds = xr.Dataset(data_vars=dict(lat_wind_mean=(["latitude","time"], lat_wind_mean)))
lat_wind_mean0 = ds.lat_wind_mean
lat_wind_mean1 = lat_wind_mean0.sortby('time')
lat_wind_mean2 = Mann_Kendall_test(lat_wind_mean1,'time',MK_modified=True,method="linregress",alpha=0.05,
                                coords_name = {'time':'time','x':'latitude'})
lat_wind_mean_trend = lat_wind_mean2.compute()
#%%%%%%%%%%%%%%%%%% trends of u10_zonal_NH %%%%%%%%%%%%%%%%%%
ds = xr.Dataset(data_vars=dict(u10_zonal_NH = (["latitude","time"], u10_zonal_NH)))
u10_zonal0_NH = ds.u10_zonal_NH
u10_zonal1_NH = u10_zonal0_NH.sortby('time')
u10_zonal2_NH = Mann_Kendall_test(u10_zonal1_NH,'time',MK_modified=True,method="linregress",alpha=0.05,
                                coords_name = {'time':'time','x':'latitude'})
u10_zonal_NH_trend0 = u10_zonal2_NH.compute()
u10_zonal_NH_trend  = u10_zonal_NH_trend0.trend*120
u10_zonal_NH_trend1 = u10_zonal_NH_trend0.trend*120
u10_zonal_NH_std    = u10_zonal_NH_trend0.std_error*120
u10_zonal_NH_p      = u10_zonal_NH_trend0.p.values
u10_zonal_NH_trend1[np.where(u10_zonal_NH_p > 0.05)] = np.nan
u10_zonal_NH_std[np.where(u10_zonal_NH_p    > 0.05)] = np.nan
#%%%%%%%%%%%%%%%%%% trends of u10_zonal_KC %%%%%%%%%%%%%%%%%%
ds = xr.Dataset(data_vars=dict(u10_zonal_KC = (["latitude","time"], u10_zonal_KC)))
u10_zonal0_KC = ds.u10_zonal_KC
u10_zonal1_KC = u10_zonal0_KC.sortby('time')
u10_zonal2_KC = Mann_Kendall_test(u10_zonal1_KC,'time',MK_modified=True,method="linregress",alpha=0.05,
                                coords_name = {'time':'time','x':'latitude'})
u10_zonal_KC_trend0 = u10_zonal2_KC.compute()
u10_zonal_KC_trend  = u10_zonal_KC_trend0.trend*120
u10_zonal_KC_trend1 = u10_zonal_KC_trend0.trend*120
u10_zonal_KC_std    = u10_zonal_KC_trend0.std_error*120
u10_zonal_KC_p      = u10_zonal_KC_trend0.p.values
u10_zonal_KC_trend1[np.where(u10_zonal_KC_p > 0.05)] = np.nan
u10_zonal_KC_std[np.where(u10_zonal_KC_p    > 0.05)] = np.nan
#%%%%%%%%%%%%%%%%%% trends of u10_zonal_GS %%%%%%%%%%%%%%%%%%
ds = xr.Dataset(data_vars=dict(u10_zonal_GS=(["latitude","time"], u10_zonal_GS)))
u10_zonal0_GS = ds.u10_zonal_GS
u10_zonal1_GS = u10_zonal0_GS.sortby('time')
u10_zonal2_GS = Mann_Kendall_test(u10_zonal1_GS,'time',MK_modified=True,method="linregress",alpha=0.05,
                                coords_name = {'time':'time','x':'latitude'})
u10_zonal_GS_trend0 = u10_zonal2_GS.compute()
u10_zonal_GS_trend  = u10_zonal_GS_trend0.trend*120
u10_zonal_GS_trend1 = u10_zonal_GS_trend0.trend*120
u10_zonal_GS_std    = u10_zonal_GS_trend0.std_error*120
u10_zonal_GS_p      = u10_zonal_GS_trend0.p.values
u10_zonal_GS_trend1[np.where(u10_zonal_GS_p > 0.05)] = np.nan
u10_zonal_GS_std[np.where(u10_zonal_GS_p    > 0.05)] = np.nan

# Plot the WSC overlaid with wind stress, the zonal mean wind components and the time series of the latitude of zero wind stress

In [ ]:
m       = Basemap(projection='merc',llcrnrlat=np.nanmin(lat),urcrnrlat= np.nanmax(lat),llcrnrlon=np.nanmin(lon),urcrnrlon=np.nanmax(lon),resolution='c')
x, y    = m(lon, lat)
cx,cy   = m(95, 52)
yy      = np.arange(1, y.shape[0], 24)
xx      = np.arange(1, x.shape[1], 24)
points  = np.meshgrid(yy, xx)
labels  = ['a','b','c','d','e']
levels1 = np.linspace(-2,2,50)
tick_marks1   = np.linspace(-2,2,11)
cmaps1        = cmocean.cm.balance
fig_ratio     = 0.9
fig           = plt.figure(figsize=(60, 30))
gs            = gridspec.GridSpec(4,3)
labelfont     = 40
padspacescale = 15
labelpadscale = 5
linefont      = 6
scale         = 1.0
m_scale       = 10
#%%%%%%%%%%%%%%%%%%%%%%%%%%% Wind spatial distribution %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
ax = fig.add_subplot(gs[0:2])
l, b, w, h = ax.get_position().bounds
m   = Basemap(projection='merc',llcrnrlat=np.nanmin(lat),urcrnrlat= np.nanmax(lat),llcrnrlon=np.nanmin(lon),urcrnrlon=np.nanmax(lon),resolution='l')    
m.drawmeridians(np.arange(0,360, 20),labels=[0,0,0,1],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')
m.drawparallels(np.arange(0,55, 10),labels=[1,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')  
m.drawcoastlines(color='0.1',  linewidth=0.1*linefont)
m.drawmapboundary(color='0.1', linewidth=0.1*linefont)
CB  = m.contourf(x, y, curlZ_mean,cmap=cmaps1,levels=levels1,origin='lower',extend='both')
Q   = m.quiver(x[tuple(points)],y[tuple(points)],Tx_mean[tuple(points)],Ty_mean[tuple(points)],color='gray',width=0.002,scale=2)
qk1 = plt.quiverkey(Q,0.60,0.80, 0.1, r' ',labelpos='N',color='gray',linewidth=linefont,fontproperties={'size': labelfont})
qk2 = plt.quiverkey(Q,0.595,0.802,0.1, r'0.1 N m$^{-2}$',labelpos='N',color='None',linewidth=linefont,fontproperties={'size':0.9*labelfont})
for i in range(6):
    CS1=m.contour(x, y,u10_mean,levels=[-1.5*i],linewidths=linefont,linestyles='dashed',colors='xkcd:black')

for i in range(6):
    CS2=m.contour(x, y,u10_mean,levels=[1.5*i],linewidths=linefont,linestyles='solid',colors='xkcd:black')
CS=m.contour(x, y,u10_mean,levels=[0.0],linewidths=1.0*linefont,linestyles='solid',colors='xkcd:cherry red')
# CS=m.contour(x, y,curlZ_mean,np.arange(-0.0001,0.0001,5),linewidths=linefont,linestyles='solid',colors='xkcd:cherry red')
m.plot(x[0,:],  y[0,:],linewidth=linefont, linestyle='solid', color='k')
m.plot(x[:,0],  y[:,0],linewidth=linefont, linestyle='solid', color='k')
m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
ax.set_position([l-0.1, b, 1.2*scale*w, 1.2*scale*h])   
ax.spines['left'].set_linewidth(labelpadscale)
ax.spines['top'].set_linewidth(labelpadscale)
ax.spines['right'].set_linewidth(labelpadscale)
ax.spines['bottom'].set_linewidth(labelpadscale)
plt.xlabel('Longitude',fontsize=1.0*labelfont,labelpad=5.0*padspacescale,family='sans-serif')
plt.ylabel('Latitude',fontsize=1.0*labelfont,labelpad=10*padspacescale,family='sans-serif')
plt.text(cx, cy,labels[0],color='xkcd:black', fontsize=labelfont, weight='bold',family='sans-serif')
m.fillcontinents(color='0.9', lake_color='white')
cbaxes1 = fig.add_axes([0.125, 0.63, 0.4, 0.02])
cb1 = plt.colorbar(CB,orientation='horizontal',cax = cbaxes1)
cb1.set_ticks(tick_marks1)
cb1.ax.tick_params(labelsize=1.0*labelfont)
cb1.set_label(r'Wind stress curl (10$^{-7}$ N m$^{-3}$)', fontsize=labelfont,labelpad=0)
#%%%%%%%%%%%%%%%%%%%%%%%%%% Latitude distribution of U10, U10 trend and WSC %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for i in range(3):
    ax1 = fig.add_subplot(gs[i+3],facecolor='0.97')
    l, b, w, h = ax1.get_position().bounds
    if i==0:
        l = l+0
        u10_zonal_trend  = u10_zonal_NH_trend    
        u10_zonal_trend1 = u10_zonal_NH_trend1  
        u10_zonal_std    = u10_zonal_NH_std
        mean_u10         = mean_u10_NH
        mean_curlZ       = mean_curlZ_NH
        loc_lables       = 'NH'
        loc_colors       = 'xkcd:black'
    elif i==1:
        l = l-0.125
        u10_zonal_trend  = u10_zonal_KC_trend    
        u10_zonal_trend1 = u10_zonal_KC_trend1  
        u10_zonal_std    = u10_zonal_KC_std
        mean_u10         = mean_u10_KC
        mean_curlZ       = mean_curlZ_KC
        loc_lables       = 'NP'
        loc_colors       = 'xkcd:gray'
    elif i==2:
        l=l-0.25
        u10_zonal_trend  = u10_zonal_GS_trend    
        u10_zonal_trend1 = u10_zonal_GS_trend1  
        u10_zonal_std    = u10_zonal_GS_std
        mean_u10         = mean_u10_GS
        mean_curlZ       = mean_curlZ_GS
        loc_lables       = 'NA'
        loc_colors       = 'xkcd:bluish'
    ########################## U10 wind trend ############################
    ax1.plot(u10_zonal_trend,  lat0, color='xkcd:blue',  linewidth=1.5*linefont,zorder=3)
    ax1.plot(u10_zonal_trend1, lat0, color='xkcd:blue', linewidth=3*linefont)
    ax1.plot([0,0], [np.min(lat0),np.max(lat0)], color='black', linewidth=1.5*linefont,linestyle='dashed')
    plt.text(-0.4,  42, loc_lables, fontsize = 1.8*labelfont, color=loc_colors,family='sans-serif')
    ax1.fill_betweenx(lat0,u10_zonal_trend-u10_zonal_std, u10_zonal_trend+u10_zonal_std ,alpha=0.3)
    ax1.set_xlabel(r'U10 trend (m s$^{-1}$ per decade)',color='xkcd:blue', fontsize=0.8*labelfont,labelpad=0)
    ax1.set_ylabel(r'Latitude',fontsize=0.8*labelfont,labelpad=10)
    ax1.set_xlim(-0.4, 0.4)
    ax1.set_xticks(np.arange(-0.4,0.4,0.2))
    ax1.set_ylim(np.min(lat0),np.max(lat0))
    ax1.axes.xaxis.set_tick_params(pad=0.8*labelpadscale)
    ax1.axes.yaxis.set_tick_params(colors='black',pad=0.8*labelpadscale)
    xminorLocator   = plt.MultipleLocator(0.1)
    xmajorLocator   = plt.MultipleLocator(0.2)
    xmajorFormatter = plt.FormatStrFormatter('%3.1f')
    ax1.xaxis.set_minor_locator(xminorLocator)
    ax1.xaxis.set_major_locator(xmajorLocator)
    ax1.xaxis.set_major_formatter(xmajorFormatter)
    plt.xticks(fontsize=0.8*labelfont)
    plt.yticks(np.linspace(0,50,6),['0$^\circ$','10$^\circ$N','20$^\circ$N','30$^\circ$N','40$^\circ$N','50$^\circ$N'],fontsize=0.8*labelfont)
    plt.grid(color ="tab:gray",linestyle='dashed', linewidth=0.5*linefont)
    ax1.spines['bottom'].set_linewidth(linefont)
    ax1.spines['left'].set_linewidth(linefont)
    ax1.spines['top'].set_linewidth(linefont)
    ax1.spines['right'].set_linewidth(linefont)        
    ax1.tick_params(axis='x',which='minor',bottom='on',left='on',length=6.0,width=1.0*linefont,colors='xkcd:blue',direction='out')
    ax1.tick_params(axis='x',which='major',bottom='on',left='on',length=10.0,width=1.0*linefont,colors='xkcd:blue',direction='out')
    ax1.spines['top'].set_color('xkcd:forest green')
    ax1.spines['bottom'].set_color('xkcd:blue')
    plt.text(-0.65, 55,labels[i+1],color='xkcd:black', fontsize=labelfont, weight='bold',family='sans-serif')
    ########################## U10 wind ############################
    ax2 = ax1.twiny()
    plt.xticks(fontsize=0.8*labelfont)
    ax2.plot(mean_u10,  lat0, color='xkcd:forest green',  linewidth=1.5*linefont,zorder=3)
    ax2.set_xlabel(r'U10 (m s$^{-1}$)',color='xkcd:forest green',fontsize=0.8*labelfont,labelpad=10)
    ax2.set_xlim(-8, 8)
    ax2.set_xticks(np.arange(-8,8,2))
    ax2.spines['top'].set_color('none')
    ax2.spines['bottom'].set_color('none')
    xminorLocator   = plt.MultipleLocator(2)
    xmajorLocator   = plt.MultipleLocator(4)
    xmajorFormatter = plt.FormatStrFormatter('%3.1f')
    ax2.xaxis.set_minor_locator(xminorLocator)
    ax2.xaxis.set_major_locator(xmajorLocator)
    ax2.xaxis.set_major_formatter(xmajorFormatter)
    ax2.axes.xaxis.set_tick_params(pad=0.8*labelpadscale)
    ax2.tick_params(axis='x',which='minor',top='on',length=6.0,width=1.0*linefont,colors='xkcd:forest green',direction='out')
    ax2.tick_params(axis='x',which='major',top='on',length=10.0,width=1.0*linefont,colors='xkcd:forest green',direction='out')
    ax2.tick_params(axis='y',which='major',top='on',length=10.0,width=1.0*linefont,colors='xkcd:black',direction='out')
    ########################## Wind stress curl ############################
    ax3 = ax1.twiny()
    ax3.plot(mean_curlZ, lat0, color='xkcd:blue purple',  linewidth=1.5*linefont,zorder=3)
    ax3.set_xlim(-0.8e-7, 0.8e-7)
    ax3.set_xticks(np.arange(-0.8e-7, 0.8e-7,0.4e-7))
    ax3.axes.xaxis.set_ticklabels([]) 
    ax3.spines['top'].set_color('none')
    ax3.spines['bottom'].set_color('none')    
    ax1.set_position([l, b-0.22, 0.45*scale*w, 1.4*scale*h]) 
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    ax4 = fig.add_subplot(gs[i+6],facecolor='none')
    l, b, w, h = ax4.get_position().bounds
    ax4.set_xlabel(r'Wind Stress Curl (10$^{-7}$ N m$^{-3}$)',color='xkcd:blue purple',fontsize=0.8*labelfont,labelpad=0)
    ax4.set_xlim(-0.8e-7, 0.8e-7)
    ax4.set_ylim(np.min(lat0),np.max(lat0))
    plt.xticks(np.linspace(-0.8e-7,0.8e-7,5), ['-0.8', '-0.4','0', '0.4','0.8'],fontsize=0.8*labelfont)
    plt.xticks(fontsize=0.8*labelfont)
    plt.yticks(fontsize=0*labelfont)
    ax4.axes.xaxis.set_tick_params(pad=0.8*labelpadscale)
    ax4.tick_params(axis='x',which='minor',bottom='on',length=5.0,width=1.0*linefont,colors='xkcd:blue purple',direction='out')
    ax4.tick_params(axis='x',which='major',bottom='on',length=10.0,width=1.0*linefont,colors='xkcd:blue purple',direction='out')
    ax4.tick_params(axis='y',which='major',right='off',length=10.0,width=0*linefont,colors='none',direction='out')
    ax4.axes.yaxis.set_ticklabels([])  
    ax4.spines['bottom'].set_linewidth(linefont)
    ax4.spines['bottom'].set_color('xkcd:blue purple')
    ax4.spines['top'].set_color('none')
    ax4.spines['left'].set_color('none')
    ax4.spines['right'].set_color('none')
    if i==0:
        l=l+0.0
    elif i==1:
        l = l-0.125
    elif i==2:
        l=l-0.25
    ax4.set_position([l, b-0.07, 0.45*scale*w, 1.4*scale*h]) 
#%%%%%%%%%%%%%%%%%%%%%%%%%% Time series of SAM index %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    ax5 = fig.add_subplot(gs[9:11],facecolor='0.97')
    l, b, w, h = ax5.get_position().bounds
    ax5.plot(t_range, latitudes_0_u10_NH, color='xkcd:black', linewidth=1.5*linefont)
    ax5.plot(t_range, latitudes_0_u10_KC, color='xkcd:gray', linewidth=1.5*linefont)
    ax5.plot(t_range, latitudes_0_u10_GS, color='xkcd:bluish', linewidth=1.5*linefont)
    ax5.plot(t_range, latitudes_0_u10_trend_NH, color='xkcd:black', linewidth=1.5*linefont,linestyle='dashed')
    ax5.plot(t_range, latitudes_0_u10_trend_KC, color='xkcd:gray', linewidth=1.5*linefont,linestyle='dashed')
    ax5.plot(t_range, latitudes_0_u10_trend_GS, color='xkcd:bluish', linewidth=1.5*linefont,linestyle='dashed')
    ax5.plot([t_range[20], t_range[50]], [34.0,34.0], color='xkcd:black', linewidth=1.5*linefont)
    ax5.plot([t_range[140], t_range[170]], [34.0,34.0], color='xkcd:gray', linewidth=1.5*linefont)
    ax5.plot([t_range[260], t_range[290]], [34.0,34.0], color='xkcd:bluish', linewidth=1.5*linefont)
    plt.text(t_range[55],  33.7, 'NH', fontsize = 1.8*labelfont, color='xkcd:black',family='sans-serif')
    plt.text(t_range[175], 33.7, 'NP', fontsize = 1.8*labelfont, color='xkcd:gray',family='sans-serif')
    plt.text(t_range[295], 33.7, 'NA', fontsize = 1.8*labelfont, color='xkcd:bluish',family='sans-serif')
    ax5.set_ylabel(r'Latitude',fontsize=1.0*labelfont,labelpad=5,color='xkcd:black')
    ax5.set_ylim(28.5,34.5)
    plt.xlim(t_range[0]-6,t_range[-1]+5)
    xminorLocator   = mpl.dates.YearLocator(1)
    xmajorLocator   = mpl.dates.YearLocator(2)
    xmajorFormatter = mpl.dates.DateFormatter('%Y')
    ax5.axes.xaxis.set_tick_params(pad=0.8*labelpadscale)
    ax5.yaxis.set_label_position("left")
    ax5.axes.yaxis.set_tick_params(colors='black',pad=0.8*labelpadscale)
    ax5.axes.xaxis.set_ticklabels([]) 
    ax5.xaxis.set_minor_locator(xminorLocator)
    ax5.xaxis.set_major_locator(xmajorLocator)
    ax5.xaxis.set_major_formatter(xmajorFormatter)
    plt.yticks(fontsize=1.0*labelfont)
    plt.xticks(fontsize=1.0*labelfont,rotation=45)
    plt.grid(color ="tab:gray",linestyle='dashed', linewidth=0.5*linefont)
    ax5.set_yticks(np.linspace(29,34,6))
    ax5.set_yticklabels(['29$^\circ$N','30$^\circ$N','31$^\circ$N','32$^\circ$N','33$^\circ$N','34$^\circ$N'])
    ax5.axes.yaxis.set_tick_params(colors='black',pad=0.8*labelpadscale)
    ax5.spines['bottom'].set_linewidth(linefont)
    ax5.spines['left'].set_linewidth(linefont)
    ax5.spines['top'].set_linewidth(linefont)
    ax5.spines['right'].set_linewidth(linefont)    
    ax5.tick_params(axis='x',which='minor',bottom='on',length=10.0,width=1.0*linefont,colors='xkcd:black',direction='out')
    ax5.tick_params(axis='x',which='major',bottom='on',length=20.0,width=1.0*linefont,colors='xkcd:black',direction='out')
    ax5.tick_params(axis='y',which='minor',right='on',length=10.0,width=1.0*linefont,colors='xkcd:black',direction='out')
    ax5.tick_params(axis='y',which='major',right='on',length=20.0,width=1.0*linefont,colors='xkcd:black',direction='out')
    ax5.set_xlabel('Year',fontsize=1.0*labelfont,labelpad=10,family='sans-serif')
    plt.yticks(fontsize=1.0*labelfont) 
    plt.text(t_range[0]-42, 34.8,labels[4],color='xkcd:black', fontsize=labelfont, weight='bold',family='sans-serif')
    ax5.set_position([l+0.0, b-0.2, 0.8*scale*w, 1.5*scale*h]) 
fig.savefig(directory+'/Figures/Figure5_wind.png',dpi=300,bbox_inches = 'tight')